# Purpose

This notebooks graphs the performance results of 5G core networks. The traffic for the 5G core networks is generated using a [5G core traffic generator](https://github.com/tariromukute/core-tg). The performance results are collected using a bcc and bpftrace tools.

In [1]:
# configure spark variables
from pyspark.context import SparkContext
from pyspark.sql.context import SQLContext
from pyspark.sql.session import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
    
sc = SparkContext()
sqlContext = SQLContext(sc)
spark = SparkSession(sc)

# load up other dependencies
import re
import pandas as pd

import glob
import matplotlib.pyplot as plt
import numpy as np

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/08/21 21:05:10 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/08/21 21:05:10 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/08/21 21:05:10 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
/Users/tariromukute/Documents/phd/opengilan/.venv/lib/python3.11/site-packages/pyspark/sql/context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


23/08/21 21:05:26 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [2]:
import os
if not os.path.exists("images"):
    os.mkdir("images")

import os
import glob
import plotly.express as px
from plotly.subplots import make_subplots
from pyspark.sql.types import StructType,StructField, StringType, IntegerType
from pyspark.sql.functions import expr
basePath = "../results"

In [4]:
html_output_file = '../open5gs.html'
with open(html_output_file, 'w') as f:
    f.write('<h1>Open GiLAN Testbed Results</h1>')
    f.write('<h3> The graphs summaries the NFV performance metrics<h3>')
    f.write('<h4> General Workload Chacterisation </h4>')
    f.write('<a href="#system-chaterisation"> Skip to results </a></h5>')
    f.write('<p> The system run different processes depending on the applications running. The operations of these applications and their respective processes is \
        execute through system calls. There are wide range of system calls that can be run by the OS. In general the frequent types of system calls can provide \
        a general chaterisationof the workload running on the OS. The workload charaterisation is a good starting point in understanding the system or applications \
        running the system. In addition to the frequent system calls, details on the processes making syscalls is helpful in understanding the system. \
        \
        The latency of the both the system calls and the processes making system calls is a starting point in understand the latency of the system as a whole. From these \
        results we can go further to look at the performance results of the different compute resources. The chaterisations helps in knowing the compute results to focus on \
        e.g., if there is a load or read syscalls we can focus on Filesystem and cache.</p>')
    f.write('<h4> CPU  <h4>')
    f.write('<h5><a href="#cpu-metrics"> Skip to results </a></h5>')
    f.write('<p> The CPU is responsible for executing all workloads on the NFV. Like other resources, the CPU is managed by the kernel. The user-level applications access CPU resources by sending system calls to the kernel. The kernel also receives other system call requests from different processes; memory loads and stores can issue page faults system calls. The primary consumers of CPU resources are threads (also called tasks), which belong to procedures, kernel routines and interrupt routes. The kernel manages the sharing via a CPU scheduler.</p>')
    f.write('<p> There are three thread states: ON-PROC for threads running on a CPU, RUNNABLE for threads that could run but are waiting their turn, and SLEEP for blocked lines on another event, including uninterruptible waits. These can be categorised into two for more accessible analysis, on-CPU referring to ON-PROC, and off-CPU referring to all other states, where the thread is not running on a CPU. Lines leave the CPU in one of two ways: (1) voluntary if they block on I/O, a lock, or asleep, or (2) involuntary if they have exceeded their scheduled allocation of CPU time. When a CPU switches from running one process or thread to another, it switches address spaces and other metadata. This process is called context switching; it also consumes the CPU resources. All these processes, described, in general, consume the CPU time. In addition to the time, another CPU resource used by the methods, kernel routines and interrupts routines is the CPU cache.</p>')
    f.write('<p> There are typically multiple levels of CPU cache, increasing in both size and latency. The caches end with the last-level store (LLC), large (Mbytes) and slower. On a processor with three levels of supplies, the LLC is also the Level 3 cache. Processes are instructions to be interpreted and run by the CPU. This set of instructions is typically loaded from RAM and cached into the CPU cache for faster access. The CPU first checks the lower cache, i.e., L1 cache. If the CPU finds the data, this is called a hit. If the CPU does not see the data, it looks for it in L2 and then L3. If the CPU does not find the data in any memory caches, it can access it from your system memory (RAM). When that happens, it is known as a cache miss. In general, a cache miss means high latency, i.e., the time needed to access data from memory. </p>')

    f.write('<h4> Memory <h4>')
    f.write('<h5><a href="#memory-metrics"> Skip to results </a> </h5>')
    f.write('<p> The kernel and processor are responsible for mapping the virtual memory to physical memory. For efficiency, memory mappings are created in groups of memory called <em>pages</em>. When an application starts, it begins with a request for memory allocation. In the case that there is no free memory on the heap, the syscall <em>brk()</em> is issued to extend the size of the bank. However, if there is free memory on the heap, a new memory segment is created via the <em>mmap()</em> syscall. Initially, this virtual memory mapping does not have a corresponding physical memory allocation. Therefore when the application tries to access this allocated memory segment, the error called <em>page fault</em> occurs on the MMU. The kernel then handles the page fault, mapping from the virtual to physical memory. The amount of physical memory allocated to a process is called resident set size (RSS). When there is too much memory demand on the system, the kernel page-out daemon (kswapd) may look for memory pages to free. Three types of pages can be released in their order: pages that we read but not modified (backed by disk) these can be immediately rid, pages that have been modified (dirty) these need to be written to disk before they can be freed and pages of application memory (anonymous) these must be stored on a swap device before they can be released. kswapd, a page-out daemon, runs periodically to scan for inactive and active pages with no memory to free. It is woken up when free memory crosses a low threshold and goes back to sleep when it crosses a high threshold. Swapping usually causes applications to run much more slowly.</p>')

    f.write('<h4>Filesytem <h4>')
    f.write('<h5><a href="#filesystem-metrics"> Skip to results </a> </h5>')
    f.write('<p> The file system that applications usually interact with directly and file systems can use caching, read-ahead, buffering, and asynchronous I/O to avoid exposing disk I/O latency to the application. Logical I/O describes requests to the file system. If these requests must be served from the storage devices, they become physical I/O. Not all I/O will; many logical read requests may be returned from the file system cache and never become physical I/O. File systems are accessed via a virtual file system (VFS). It provides operations for reading, writing, opening, closing, etc., which are mapped by file systems to their internal functions. Linux uses multiple caches to improve the performance of storage I/O via the file system. These are Page cache: This contains virtual memory pages and enhances the performance of file and directory I/O. Inode cache, which are data structures used by file systems to describe their stored objects. The directory cache caches mappings from directory entry names to VFS inodes, improving the performance of pathname lookups. The page cache grows to be the largest of all these because it caches the contents of files and includes “dirty” pages that have been modified but not yet written to disk.</p>')

    f.write('<h4>Disk I/O <h4>')
    f.write('<h5><a href="#disk-metrics"> Skip to results </a> </h5>')
    f.write('<p> Linux exposes rotational magnetic media, flash-based storage, and network storage as storage devices. Therefore, disk I/O refers to I/O operations on these devices. Disk I/O is a common source of performance issues because I/O latency on storage devices is orders of magnitude slower than the nanosecond or microsecond speed of CPU and memory operations. Block I/O refers to device access in blocks. I/O is queued and scheduled in the block layer. The wait time is spent in the block layer scheduler queues and device dispatcher queues from the operating system. Service time is the time from device issue to completion. This may include the time spent waiting in an on-device line. Request time is the overall time from when an I/O was inserted into the OS queues to its completion. The request time matters the most, as that is the time that applications must wait if I/O is synchronous.</p>')

    f.write('<h4>Networking<h4>')
    f.write('<h5><a href="#networking-metrics"> Skip to results </a> </h5>')
    f.write('<p> Networking is a complex part of the Linux system. It involves many different layers and protocols, including the application, protocol libraries, syscalls, TCP or UDP, IP, and device drivers for the network interface. In general, the Networking system can be broken down into four. The NIC and Device Driver Processing first reads packets from the NIC and puts them into kernel buffers. Besides the NIC and Device driver, this process includes the DMA and particular memory regions on the RAM for storing receive and transmit packets called rings and the NAPI system for poling packets from these rings to the kernel buffers. It also incorporates some early packet processing hooks like XDP and AF\_XDP and can have custom drivers that bypass the kernel (i.e., the following two processes) like DPDK. Following is the Socket processing. This part also includes queuing and different queuing disciplines. It also incorporates some packet processing hooks like TC, Netfilter etc., which can alter the flow of the networking stack. After that is the  Protocol processing layer, which applies functions for different IP and transport protocols, both these protocols run under the context of SoftIrq. Lastly is the application process. The application receives and sends packets on the destination socket</p>')
    
    f.write('<h4>Flame Graphs to analyse code paths<h4>')
    f.write('<h5><a href="#flame-graphs"> Skip to results </a> </h5>')
    f.write('<p> A flame graph visualizes a distributed request trace and represents each service call that occurred during the requests execution path with a timed, color-coded, horizontal bar. Flame graphs for distributed traces include error and latency data to help developers identify and fix bottlenecks in their applications..</p>')

In [5]:
# General chaterisation
import plotly; print(plotly.__version__)

5.15.0


In [16]:
""" This shows how the usage syscalls change as the load changes. This can tell us:
1. How the core network is architectures to respond to increasing load
2. Comparing can tell us the core network that sends more time on syscalls. We can use that to corellate to the performance of the core network

"""

top_n = 5

syscount_df = spark.read.option("basePath", basePath).json(
f"{basePath}/cn=*/ues=*/tool=syscount")

# Group DataFrame by x-labels, find max y-values in each group,
# then sort resulting Series in descending order
label_maxes = syscount_df.toPandas().groupby(['syscall', 'cn', 'ues'])['time (ms)'].max().sort_values(ascending=False)

# Select the top n labels with the highest y-values
top_labels = label_maxes.head(top_n).index.tolist()

print(top_labels)

stats_df = syscount_df.toPandas()

df_syscount = stats_df[stats_df['syscall'].isin(top_labels)].sort_values('ues')

df_syscount = syscount_df.toPandas().groupby(['cn', 'ues']).agg({ 'count': 'sum', 'time (ms)': 'sum' }).reset_index()

syscount_fig = px.line(df_syscount,
                x="ues", y="time (ms)", color="cn",
                labels={
                     "ues": "Number of UEs",
                     "time (ms)": "Time (ms)",
                     "syscall": "System calls",
                     "count": "Number of calls",
                     "cn": "Core Network"
                 },
                title='Syscalls across the system (by latency)',
                markers=True)
# syscount_fig.update_traces(textinfo='value')
# syscount_fig.update_traces(textinfo='time (us)')
syscount_fig.show()
syscount_fig.write_image("images/syscount_fig_m2.medium.jpeg")

# Group DataFrame by x-labels, find max y-values in each group,
# then sort resulting Series in descending order
label_maxes = syscount_df.toPandas().groupby('syscall')['count'].max().sort_values(ascending=False)

# Select the top n labels with the highest y-values
top_labels = label_maxes.head(top_n).index.tolist()

sysprocess_count_fig = px.line(df_syscount,
                x="ues", y="count", color="cn",
                hover_data=["count", "time (ms)"],
                labels={
                     "ues": "Number of UEs",
                     "time (ms)": "Time (ms)",
                     "syscall": "System calls",
                     "count": "Number of calls",
                     "cn": "Core network"
                },
                title=f'Processes making syscall (by number of calls)',
                markers=True)
# sysprocess_count_fig.update_traces(textinfo='value')
sysprocess_count_fig.show()
sysprocess_count_fig.write_image(f"images/syscount_count_fig_m2.medium.jpeg")
with open(html_output_file, 'a') as f:
    f.write('<h2 id="syscall-count"> Syscall counts across the system, with latency information. </h2>')
    f.write('<p>futex: (short for “fast userspace mutex”) is a kernel system call that programmers can use to implement basic locking, \
            or as a building block for higher-level locking abstractions such as semaphores and POSIX mutexes or condition variables \
            A known cause of high futex system calls on VM is high contention on shared-memory resources that causes many threads \
            to wait on futexes <a href="https://access.redhat.com/solutions/534663">source</a>. </p>')
    f.write('<p>o_getevents: system calls are used to read asynchronous I/O events from the completion queue of an AIO context. \
            An AIO context is a data structure that holds information about pending and completed I/O operations \
            <a href="https://linux.die.net/man/2/io_getevents">source</a>. \
            \
            The io_getevents() system call is Linux-specific and hence affects portability of programs.</p>')
    f.write('<p>poll_wait: is a system call that waits for events on an epoll instance, which is a mechanism \
            for monitoring multiple file descriptors for I/O readiness.</p>')
    f.write(syscount_fig.to_html(full_html=False, include_plotlyjs='cdn'))
    f.write(sysprocess_count_fig.to_html(full_html=False, include_plotlyjs='cdn'))

[('futex', 'oai', 2), ('futex', 'oai', 20), ('futex', 'oai', 200), ('futex', 'oai', 2000), ('epoll_wait', 'oai', 20)]


In [ ]:
""" 

"""

In [15]:
sysprocess_df = spark.read.option("basePath", basePath).json(
f"{basePath}/cn=*/ues=*/tool=sysprocess")

# Group DataFrame by x-labels, find max y-values in each group,
# then sort resulting Series in descending order
label_maxes = sysprocess_df.toPandas().groupby('comm')['time (ms)'].max().sort_values(ascending=False)

# Select the top n labels with the highest y-values
top_labels = label_maxes.head(top_n).index.tolist()

stats_df = sysprocess_df.toPandas()

sysprocess_fig = px.line(stats_df[stats_df['comm'].isin(top_labels)].sort_values('ues'),
                x="ues", y="time (ms)", color="comm",
                hover_data=["count", "time (ms)"],
                labels={
                     "ues": "Number of UEs",
                     "time (ms)": "Time (ms)",
                     "syscall": "System calls",
                     "count": "Number of calls",
                     "comm": "Process name"
                },
                title=f'Processes making syscall (by latency)',
                markers=True)
# sysprocess_fig.update_traces(textinfo='value')
sysprocess_fig.show()
sysprocess_fig.write_image(f"images/sysprocess_fig_m2.medium.jpeg")
with open(html_output_file, 'a') as f:
    f.write(f'<h2 id="syscall-processes"> Processes are making syscalls with latency information </h2>')
    f.write('<p>  </p>') 
    f.write(sysprocess_fig.to_html(full_html=False, include_plotlyjs='cdn'))

# Group DataFrame by x-labels, find max y-values in each group,
# then sort resulting Series in descending order
label_maxes = sysprocess_df.toPandas().groupby('comm')['count'].max().sort_values(ascending=False)

# Select the top n labels with the highest y-values
top_labels = label_maxes.head(top_n).index.tolist()

stats_df = sysprocess_df.toPandas()

sysprocess_count_fig = px.line(stats_df[stats_df['comm'].isin(top_labels)].sort_values('ues'),
                x="ues", y="count", color="comm",
                hover_data=["count", "time (ms)"],
                labels={
                     "ues": "Number of UEs",
                     "time (ms)": "Time (ms)",
                     "syscall": "System calls",
                     "count": "Number of calls",
                     "comm": "Process name"
                },
                title=f'Processes making syscall (by number of calls)',
                markers=True)
# sysprocess_count_fig.update_traces(textinfo='value')
sysprocess_count_fig.show()
sysprocess_count_fig.write_image(f"images/sysprocess_count_fig_m2.medium.jpeg")
with open(html_output_file, 'a') as f:
    f.write(f'<h2 id="syscall-count-processes"> Processes are making syscalls by number of calls</h2>')
    f.write('<p>  </p>') 
    f.write(sysprocess_count_fig.to_html(full_html=False, include_plotlyjs='cdn'))

In [12]:
""" For each of the syscalls, we plot for the core networks, the amount of times and occurances. This can tell us:
1. How the core network are architected to respond to load for different operations e.g., how their socket read logic is implemented
to work and how that responses to change in traffic load
2. Relative to other core network which syscalls it uses the most. For example this can tell us the syscall that has the most differentiating factor, 
e.g., if all syscalls are relatively the same and there is a huge difference for sched_yield, then it is likely the differentiating syscall or design

"""

syscalls = ['futex', 'clock_nanosleep', 'epoll_wait', 'poll', 'ppoll', 'recvmsg', 'recvfrom', 
            'sendto', 'read', 'select', 'sched_yield', 'write', 'nanosleep', 'epoll_pwait']

# syscalls = ['sched_yield']

top_n = 5
# Run for each syscall
for syscall in syscalls:
    sysprocess_df = spark.read.option("basePath", basePath).json(
    f"{basePath}/cn=*/ues=*/tool=sysprocess_{syscall}")

    # label_maxes = sysprocess_df.toPandas().groupby('comm')['time (ms)'].max().sort_values(ascending=False)

    # # Select the top n labels with the highest y-values
    # top_labels = label_maxes.head(top_n).index.tolist()

    # stats_df = sysprocess_df.toPandas()

    cn_df = sysprocess_df.toPandas().groupby(['cn', 'ues']).agg({ 'count': 'sum', 'time (ms)': 'sum' }).reset_index()

    sysprocess_fig = px.line(cn_df.sort_values('ues'),
                    x="ues", y="time (ms)", color="cn", 
                    hover_data=["count", "time (ms)"],
                    labels={
                        "ues": "Number of UEs",
                        "time (ms)": "Time (ms)",
                        "syscall": "System calls",
                        "count": "Number of calls",
                        "cn": "Core Network"
                    },
                    title=f'Core network syscall {syscall} (by latency)',
                    markers=True)
    # sysprocess_fig.update_traces(textinfo='value')
    sysprocess_fig.show()
    sysprocess_fig.write_image(f"images/sysprocess_{syscall}_fig_m2.medium.jpeg")
    with open(html_output_file, 'a') as f:
        f.write(f'<h2 id="{syscall}-syscall-processes"> Processes are making {syscall} syscalls with latency information </h2>')
        f.write('<p>  </p>') 
        f.write(sysprocess_fig.to_html(full_html=False, include_plotlyjs='cdn'))
    
    label_maxes = sysprocess_df.toPandas().groupby('comm')['count'].max().sort_values(ascending=False)

    # Select the top n labels with the highest y-values
    top_labels = label_maxes.head(top_n).index.tolist()

    stats_df = sysprocess_df.toPandas()

    sysprocess_count_fig = px.line(cn_df.sort_values('ues'),
                    x="ues", y="count", color="cn",
                    hover_data=["count", "time (ms)"],
                    labels={
                        "ues": "Number of UEs",
                        "time (ms)": "Time (ms)",
                        "syscall": "System calls",
                        "count": "Number of calls",
                        "cn": "Core network"
                    },
                    title=f'Processes making {syscall} syscall (by number of calls)',
                    markers=True)
    # sysprocess_count_fig.update_traces(textinfo='value')
    sysprocess_count_fig.show()
    sysprocess_count_fig.write_image(f"images/sysprocess_count_{syscall}_fig_m2.medium.jpeg")
    with open(html_output_file, 'a') as f:
        f.write(f'<h2 id="{syscall}-syscall-count-processes"> Processes are making {syscall} syscalls by number of calls</h2>')
        f.write('<p>  </p>') 
        f.write(sysprocess_count_fig.to_html(full_html=False, include_plotlyjs='cdn'))